##**Assignment 3**

Name: Hazel Mbambo

DTSC 680 Mushroom Classifier



Overview:
The purpose of this assignment is for you to gain experience with the KNN and PCA
algorithms, as well as to assess your ability to construct a well-organized and
well-documented notebook. In this assignment you will not receive a notebook template.
Upon submission, be sure that your notebook is well documented - this is required for
code development and collaboration in industry, and is needed for grading your work.
You will be heavily graded on the clarity of your notebook. How easy is it to understand
the meaning of your computations? Do the names of your variables clearly
communicate their meaning? Is your notebook documented completely?
Document/Comment your code so that someone who didn’t know about your
assignment would be able to follow what you were doing every step of the way.

####**Preliminaries**
Let's import some common packages:

In [ ]:
# Common imports
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import numpy as np
import pandas as pd
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import os

#encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

####**1. Import the Mushroom data set.**

Getting the columns from the data set and putting the data and names together.

From the readme file you can see that class is the response data. P = Poisonus and e = edible

In [ ]:
#column names
names = ['class','cap-shape', 'cap-surface', 'cap-color', 'bruises?', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat'] 


In [ ]:
#import data
data = pd.read_csv("agaricus-lepiota.data", names=names)

#View data
data

,class,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8120,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8121,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8122,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l


In [ ]:
#one-hot encoding before splitting features and response ?

####**2. Use the KNN algorithm to impute missing values in the dataset.** 

The first step is to think through what will be our feature data and what will be our
response data for this imputation step. Concluded that because stalk-root is the only one with missing rows, it is what we need to use as the response data for this imputing step

In [ ]:
#view missing data
data.isnull().sum().sum()

0

In [ ]:
#There should be missing data per: Missing Attribute Values: 2480 of them (denoted by "?"), all for attribute #11./(11. stalk-root:  )
#Converting '?' to NaN
df = data.replace('?',np.NaN)

In [ ]:
#checking if missing values equal 2480
df.isnull().sum().sum()

2480

In [ ]:
#Spliting those with missing data from the features
data_no_na = data[data['stalk-root']!='?'] 
data_na = data[data['stalk-root']=='?'] 

####**Encoding** 

Found code on stackover flow that seems to help us with this as it was giving me trouble. The columns_enc will be used a few steps below to relable the data

In [ ]:
#columns we will need 
columns_enc=['cap-shape','cap-surface','cap-color','bruises?',
           'odor','gill-attachment','gill-spacing','gill-size','gill-color',
          'stalk-shape','stalk-surface-above-ring','stalk-surface-below-ring',
          'stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color',
          'ring-number','ring-type','spore-print-color','population','habitat']

In [ ]:
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()

le1.fit(data_no_na['class']) 
le2.fit(data_no_na['stalk-root']) 



LabelEncoder()

In [ ]:
# encoding with One Hot encoding except response data
#X = data_One_hot_encoded
data_hot_enc = pd.get_dummies(data_no_na,columns=columns_enc)
# class is a output lebel, we need to encode it with labelencoder
data_hot_enc['class'] = le1.transform(data_no_na['class'])


In [ ]:
# all the missing values are in attribute stalk-root, encoding it with labelencoder
data_hot_enc['stalk-root'] = le2.transform(data_no_na['stalk-root'])
X = data_hot_enc.loc[:,~data_hot_enc.columns.isin(['class','stalk-root'])]
y = data_hot_enc['stalk-root']

######**Train your KNN model on instances**
Next, we train our KNN model on those instances that don’t have missing values, then have the model make predictions for those instances that have missing values. This is how we will have the KNN model impute missing data

In [ ]:
#though not advissed in the assigment instructions, this was where I was trpping up and splotting this way helped 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5) 

######**Fitting the models**

This is to predict the stalk root 

In [ ]:
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
y_pred

array([0, 2, 2, ..., 0, 0, 0])

####**Imputing the data**
This step adds the model's predictions into the na's  dataframe, overwritting the missing values

In [ ]:
# impute the missing values 
enc = OneHotEncoder(handle_unknown='ignore')
data_no_na_X = data_no_na.loc[:,~data_no_na.columns.isin(['class','stalk-root'])]
enc.fit(data_no_na_X)
data_na_X = data_na.loc[:,~data_na.columns.isin(['class','stalk-root'])]
X_na = enc.transform(data_na_X).toarray()
y_na_pred = knn.predict(X_na)
stalk_root_na=le2.inverse_transform(y_na_pred)
data_na['stalk-root'] = stalk_root_na

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


######PRINT MISSING VALUES

In [ ]:
stalk_root_na

array(['b', 'b', 'e', ..., 'e', 'b', 'e'], dtype=object)

In [ ]:
missing_values = data_na['stalk-root']
print(missing_values.head(10))

3984    b
4023    b
4076    e
4100    b
4104    b
4196    b
4200    b
4283    b
4291    b
4326    b
Name: stalk-root, dtype: object


In [ ]:
#making sure all null vals are imputed
data_na.isna().any()

class                       False
cap-shape                   False
cap-surface                 False
cap-color                   False
bruises?                    False
odor                        False
gill-attachment             False
gill-spacing                False
gill-size                   False
gill-color                  False
stalk-shape                 False
stalk-root                  False
stalk-surface-above-ring    False
stalk-surface-below-ring    False
stalk-color-above-ring      False
stalk-color-below-ring      False
veil-type                   False
veil-color                  False
ring-number                 False
ring-type                   False
spore-print-color           False
population                  False
habitat                     False
dtype: bool

####Putting DF back together

In [ ]:
X_na_ohe = pd.DataFrame(X_na,columns=X.columns)
X_na_ohe['stalk-root'] = y_na_pred
df2 = pd.concat([X_na_ohe,data_hot_enc])

df2.reset_index(drop=True,inplace=True)
df2['stalk-root']

0       0
1       0
2       2
3       0
4       0
       ..
8119    0
8120    0
8121    0
8122    1
8123    1
Name: stalk-root, Length: 8124, dtype: int64

##**Train a RandomForestClassifier**

3. Train a RandomForestClassifier as well as a LogisticRegression model to predict
whether a mushroom is edible or poisonous given this data set of nominally-valued
characteristics. Train the model on the feature data supplied to you after you’ve one-hot
encoded it. You should label encode the response data

In [ ]:
df2

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,stalk-root,class
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,NaN
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2,NaN
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0,NaN
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
8120,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
8121,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
8122,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,1.0


####**Train a Random and logistic Classifiers**

#### **Train Test Split**

To get our data split for the required models

In [ ]:
#googled error I was recieving and the below code seems to have corrected it
def clean_dataset(df2):
    assert isinstance(df2, pd.DataFrame), "df needs to be a pd.DataFrame"
    df2.dropna(inplace=True)
    indices_to_keep = ~df2.isin([np.nan, np.inf, -np.inf]).any(1)
    return df2[indices_to_keep].astype(np.float64)
df2 = clean_dataset(df2)
df2.reset_index(drop=True,inplace=True)
print(len(df2))

5644


In [ ]:
# dividing the datasets into two parts i.e. training datasets and test datasets
X,y = df2[df2.columns[:-1]],df2['class']
# Spliting arrays or matrices into random train and test subsets
# i.e. 70 % training dataset and 30 % test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30,random_state=42)

In [ ]:
# nominally-valued characteristics? #might use if the above suggestion found online doesnt work
# i.e. 70 % training dataset and 30 % test datasets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state=42)

**Fitting the randomforest**

Accordingly, we use the default hyperparameter settings for
both models, along with a random_state of 42 for the RandomForestClassifier.

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 100)
rnd_clf.fit(X_train, y_train)


CPU times: user 307 ms, sys: 5.67 ms, total: 313 ms
Wall time: 320 ms


RandomForestClassifier()

In [ ]:
y_pred_RF = rnd_clf.predict(X_test)

**Pred with our model**

Scores:

In [ ]:
#viewing to make sure it is hot encoded correctly as error was coming up
X_test

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,stalk-root
1694,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
4434,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3297,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3980,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4165,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3428,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3576,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1538,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0
211,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


####**Train a LogisticRegression**

Doing the same with the logistic regression 

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)


CPU times: user 118 ms, sys: 65.9 ms, total: 184 ms
Wall time: 120 ms


LogisticRegression()

**Pred with our model**

In [ ]:
y_pred_lr = log_reg.predict(X_test)

####**accuracy, precision, and recall scores**

5. Compute the accuracy, precision, and recall scores for a test set. Briefly discuss the
performance of your models in terms of these values.

In [ ]:
#prepare models
Regressor = [
    LogisticRegression(),
    RandomForestClassifier(n_estimators = 100)]

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score 
from sklearn.metrics import f1_score, precision_score, recall_score, fbeta_score
from sklearn.metrics import make_scorer, recall_score, log_loss

In [ ]:
log_cols=["Regressor", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns = log_cols)

for clf in Regressor:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    print("=" * 30)
    print(name)
    print('****Results****')
    
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    print("Accuracy: {:.4%}".format(acc))
    
    train_predictions = clf.predict(X_test)
    ll = log_loss(y_test, train_predictions)
    print("Log Loss: {}".format(ll))
    print("\n")
    
    log_entry = pd.DataFrame([[name, acc * 100, ll]], columns = log_cols)
    log = log.append(log_entry)
    
print("=" * 30)

LogisticRegression
****Results****
Accuracy: 100.0000%
Log Loss: 9.992007221626413e-16


RandomForestClassifier
****Results****
Accuracy: 100.0000%
Log Loss: 9.992007221626413e-16




##**PCA**

In [ ]:
#Check the Co-relation between features without PCA
#sns.heatmap(X.corr())
#Applying PCA
#Taking no. of Principal Components as 3
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.95,random_state=42)
pca.fit(X)
pca_data = pca.transform(X)
pca_columns = ['PC'+str(i+1) for i in range(pca_data.shape[1])] # create column names from pca_data
df_pca = pd.DataFrame(pca_data,columns=pca_columns)
df_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33
0,-1.576543,1.091614,-1.701929,-0.403847,-0.434055,0.548515,0.803155,-0.229099,-0.477424,0.765912,...,-0.045587,0.110488,0.010206,0.194444,-0.225222,0.438493,0.015333,4.059768e-15,-0.607218,0.170389
1,-0.954724,0.643852,-1.529768,-0.392295,-0.135108,-0.643373,-0.708905,-0.535166,-0.162625,-0.642925,...,0.159069,0.099999,0.308564,0.309569,-0.049249,0.027969,0.105920,7.071068e-01,-0.624161,-0.096954
2,-1.168747,0.385583,-1.666157,-0.230168,0.430590,-0.215522,-0.663655,-0.725583,0.412901,-0.637471,...,0.112663,-0.660698,0.185583,0.179146,-0.183854,0.246659,-0.027303,-7.071068e-01,0.049429,-0.407515
3,-1.471939,0.920828,-1.939439,-0.068392,-0.664457,0.500559,0.738315,0.182440,-0.038410,0.759777,...,-0.239954,0.009505,-0.165015,0.105406,-0.106291,0.275856,0.087287,1.138494e-15,0.027298,0.434757
4,-1.432947,1.979300,1.258458,-0.399312,-0.504507,-0.453058,-0.723235,0.296941,0.375556,0.283121,...,0.005679,0.036652,-0.211810,-0.238077,0.003676,0.158747,-0.271895,2.929596e-15,-0.684095,0.243574


##**Second Models**

Train two new models, a RandomForestClassifier and a LogisticRegression model,
on this reduced dataset to predict whether a mushroom is edible or not. Again, time the
training of these two models on the reduced dataset. You will again use the default
hyperparameter values for both models, with a random state parameter of 42 for the
RandomForestClassifier.

In [ ]:
%%time
# creating a RF classifier
Xpca_train, Xpca_test, y_train, y_test = train_test_split(df_pca, y, test_size = 0.30,random_state=42)
RF_pca = RandomForestClassifier(n_estimators = 100) 
# Training the model on the training dataset
# fitting RF classifier on traning data set
RF_pca.fit(Xpca_train, y_train)
# perform predictions on the test dataset
y_pred_RF_pca = RF_pca.predict(Xpca_test)
# calculate accuracy of the model
print("accuracy of the RF model: ", metrics.accuracy_score(y_test, y_pred_RF_pca))
Matrices = metrics.precision_recall_fscore_support(y_test, y_pred_RF_pca,average='macro')
print("precisio of the RF model: ", Matrices[0])
print("Recall of the RF model: ", Matrices[1])

accuracy of the RF model:  1.0
precisio of the RF model:  1.0
Recall of the RF model:  1.0
CPU times: user 1.43 s, sys: 42.8 ms, total: 1.48 s
Wall time: 1.44 s


In [ ]:
%%time
# build a LogisticRegression model
# all parameters not specified are set to their defaults
lr_pca = LogisticRegression()
# fit logisticRegression model
lr_pca.fit(Xpca_train, y_train)
# predictions on test data 
y_pred_lr_pca = lr_pca.predict(Xpca_test)
# calculate accuracy of the model
print("accuracy of the logisticRegression model: ", metrics.accuracy_score(y_test, y_pred_lr_pca))
Matrices = metrics.precision_recall_fscore_support(y_test, y_pred_lr_pca,average='macro')
print("precisio of the LogRe model: ", Matrices[0])
print("Recall of the logRe model: ", Matrices[1])

accuracy of the logisticRegression model:  0.999409681227863
precisio of the LogRe model:  0.9995278564683664
Recall of the logRe model:  0.9992138364779874
CPU times: user 67.2 ms, sys: 56.1 ms, total: 123 ms
Wall time: 73.3 ms
